In [92]:

from owlready2 import *
from rdflib import Graph, URIRef, Literal
from rdflib.plugins.sparql import prepareQuery
import networkx as nx

In [5]:
base_path='../data'
onto_path=base_path+'/brainteaser-ontology/bto.ttl'

In [47]:

brainteaser_graph=Graph().parse(onto_path, format='turtle')
brainteaser_graph.bind('bto', 'http://www.semanticweb.org/ontologies/2020/3/bto#')

In [48]:
brainteaser_graph

<Graph identifier=Nc289605f6ea348cf9b3261df3a222f28 (<class 'rdflib.graph.Graph'>)>

In [49]:

for s, p, o in brainteaser_graph:
    print(s, p, o)

http://purl.bioontology.org/ontology/UATC/A14 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
http://purl.obolibrary.org/obo/NCIT_C71410 http://www.w3.org/2000/01/rdf-schema#comment A child of your aunt or uncle or their descendents. [Definition Source: NCI]
http://purl.obolibrary.org/obo/NCIT_C89715 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/brainteaser/ontology/schema/DiseaseDisorderOrFinding
http://purl.obolibrary.org/obo/NCIT_C119640 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://purl.obolibrary.org/obo/NCIT_C26717 http://www.w3.org/2000/01/rdf-schema#comment An eyelid cyst caused by the blockage of a meibomian gland. [Definition Source: NCI]
https://w3id.org/brainteaser/ontology/schema/hasEthnicity http://www.w3.org/2000/01/rdf-schema#label hasEthnicity
http://purl.obolibrary.org/obo/NCIT_C55098 http://www.w3.org/2000/01/rdf-schema#label Serum Glutamic Pyruvic Tr

In [70]:
seed_conceepts = ["Patient", "Clinical Trial", "Disease", "Event"]
seed_concept=seed_conceepts[-1]

In [117]:
import pandas as pd

def q_to_df(q, onto: Graph = brainteaser_graph, map_n3=True):
    results=list(onto.query(q))
    # for r in results:
    #     for t in r:
    #         if isinstance(t, URIRef):
    #             print(t.n3(onto.namespace_manager))
    #         else:
    #             print(t)
    df= pd.DataFrame(results)
    if map_n3:
        df=df.map(lambda x: x.n3(onto.namespace_manager) if isinstance(x, URIRef) else x)
    return df

In [118]:
q_to_df(f"""
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                SELECT *
                WHERE {{
                     ?fun rdfs:domain bto:{seed_concept} .
                }}
                """)

,0
0,bto:consists
1,bto:hasRegisteredActivity
2,bto:hasRegisteredCondition
3,bto:pregnancy
4,bto:relapse
5,bto:trauma
6,bto:eventEnd
7,bto:eventStart


In [121]:
classes=q_to_df(f"""
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                SELECT ?cls
                WHERE {{
                     ?cls rdf:type owl:Class.
                     MINUS {{ ?cls rdfs:subClassOf ?sbcl}}.
                }}
                """, map_n3=True)
classes

,0
0,n6b4c14b82fe047db8a5b001078d71f3cb1
1,<http://www.w3.org/2000/10/swap/pim/contact#Pe...
2,bto:Activity
3,bto:Administration
4,bto:AdverseDrugReaction
5,bto:AnatomicalSite
6,bto:ClinicalTrial
7,bto:ClinicalTrialParticipation
8,bto:ClinicsAndHospitals
9,bto:Condition


In [125]:
labels = list(
    brainteaser_graph.query(
            f"""
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                SELECT ?label
                WHERE {{
                    {classes.iloc[1, 0]} rdfs:label ?label.
                }}
                """
    )
)
labels[0][0].value

'Person'

In [101]:
classes.iloc[-1, 0]

'bto:Trauma'

In [88]:
for o in brainteaser_graph.query('SELECT ?s ?p ?o WHERE {?s ?p ?o}'):
    print(o)

(rdflib.term.URIRef('http://purl.bioontology.org/ontology/UATC/A14'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual'))
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/NCIT_C71410'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'), rdflib.term.Literal('A child of your aunt or uncle or their descendents. [Definition Source: NCI]', lang='en'))
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/NCIT_C89715'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/DiseaseDisorderOrFinding'))
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/NCIT_C119640'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept'))
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/NCIT_C26717'), rdflib.term.URIRef('http://www.w3.org/2

rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range')

rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')